<a href="https://colab.research.google.com/github/toctave4/DHIS2-data-process/blob/main/burkin_search_app_fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install rapidfuzz

In [45]:
import numpy as np
import pandas as pd
import glob
from rapidfuzz import process, fuzz

In [5]:
# chargement du fichier BFA dans python
#bfa_metadata = pd.ExcelFile('/All files xls_update.xlsx')
bfa_metadata = pd.ExcelFile('/BFA Register Metadata NS 13-10-2021.xlsx')

In [6]:
# charger le fichier obtenu dans un dictionnaire python
dict_bfa_data = {}
for sht in bfa_metadata.sheet_names:
    dict_bfa_data[sht] = bfa_metadata.parse(sht)
    dict_bfa_data[sht].drop(dict_bfa_data[sht].columns[1:], axis = 1, inplace = True)
    dict_bfa_data[sht] = dict_bfa_data[sht].dropna(axis=0)

71

In [7]:
# chargement du fichier DHIS2 dans python
dhis2_metadata = pd.ExcelFile('/Burkina_DHIS2_Metadata.xlsx')

In [8]:
# charger le fichier obtenu dans un dictionnaire python
dict_DHIS2_data = {}
for sht in dhis2_metadata.sheet_names:
    dict_DHIS2_data[sht] = dhis2_metadata.parse(sht)
    dict_DHIS2_data[sht].drop(dict_DHIS2_data[sht].columns[1:], axis = 1, inplace = True)

3

In [28]:
# Creation d'un hash table contenant toutes les metadonnees sur DHIS2
hash_table_dhis2 = {}
for key in dict_DHIS2_data.keys():
    for i in range(len(dict_DHIS2_data[key])):
        hash_table_dhis2[dict_DHIS2_data[key].loc[i][0].strip().lower()] = key

In [29]:
# Creation d'un hash table contenant toutes les metadonnees du ficher BFA
hash_table_bfa = {}
for key in dict_bfa_data.keys():
    for i in range(len(dict_bfa_data[key])):
        hash_table_bfa[str(dict_bfa_data[key].iloc[i][0]).strip().lower()] = key

In [48]:
# Utilisation d'un sorted array de python pour optimiser l'acces aux donnees
bfa_array = hash_table_bfa.keys()
bfa_ordered_array = sorted(arr)

In [47]:
# Utilisation d'un sorted array de python pour optimiser l'acces aux donnees
dhis2_array = hash_table_dhis2.keys()
dhis2_ordered_array = sorted(dhis2_array)

In [49]:
dict_fuzzy_match = {'matching elements': {'BFA':{'BFA': [], 'BFA Source': []}, 'DHIS2': {'DHIS2': [], 'DHIS2 Source': []}}}
for elm_bfa in bfa_ordered_array:
    for elm_dhis2 in dhis2_ordered_array:
        if fuzz.ratio(elm_bfa, elm_dhis2) > 80: # methode ratio(str, str) du module Fuzzy permet de faire 
                                                # un matching approximatif entre 2 string en utlisant 
                                                # l'algorithm de Levenshtein
            dict_fuzzy_match['matching elements']['BFA']['BFA'].append(elm_bfa)
            dict_fuzzy_match['matching elements']['BFA']['BFA Source'].append(hash_table_bfa[elm_bfa])
            dict_fuzzy_match['matching elements']['DHIS2']['DHIS2'].append(elm_dhis2)
            dict_fuzzy_match['matching elements']['DHIS2']['DHIS2 Source'].append(hash_table_dhis2[elm_dhis2])

In [43]:
# Ecrire les resultats dans un fichier excel metadata_found.xlsx
with pd.ExcelWriter('/metadata_fuzzy_match.xlsx') as writer:
    dict_fuzzy_bfa_match_df = pd.DataFrame.from_dict(dict_fuzzy_match['matching elements']['BFA'])
    dict_fuzzy_dhis2_match_df = pd.DataFrame.from_dict(dict_fuzzy_match['matching elements']['DHIS2'])
    dict_fuzzy_match_df = pd.concat([dict_fuzzy_bfa_match_df, dict_fuzzy_dhis2_match_df], axis=1)
    dict_fuzzy_match_df.to_excel(writer, sheet_name='matching elements')